# Adding  Dropout on the  Hidden Layers

In [ ]:
# Package of interest importation

In [67]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.constraints import max_norm

We will be using the Sonar Dataset for the implemedntation/addition of dropout in our network.

In [69]:
# Fixing a random seed for reproducibility

In [70]:
seed = 7
numpy.random.seed(seed)

In [71]:
# Loading the Sonar Dataset

In [72]:
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values

In [73]:
# Splitting the dataset into X and Y variables for the model training

In [74]:
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [75]:
# Encode class values as integers

In [76]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [ ]:
# Creating a function for the network and adding a dropout in hidden layers with weight constraint.

In [77]:
def create_model():
    
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer= 'normal', activation='relu' ,kernel_constraint=max_norm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(30, kernel_initializer= 'normal', activation= 'relu', kernel_constraint=max_norm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer= 'normal', activation= 'sigmoid'))
    
    # Compile model
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss= 'binary_crossentropy' , optimizer='sgd', metrics=['accuracy'])
    
    return model

In [78]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=300, batch_size=16, verbose=0)))

In [79]:
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

In [80]:
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 54.71% (8.03%)
